# Regressionsanalys med logaritmerade variabler
_Av Anders Sundell_

[I en annan guide](logarithm_sv.html) gick vi igenom hur man skapar logaritmerade variabler, och vad de innebär. Här ska vi istället fokusera på hur man använder dem i regressionsanalyser, och vad man behöver tänka på när man ska tolka koefficienterna.

Vi ska använda oss av samma data som i det förra exemplet, det vill säga QoG Basic (version 2018). I koden nedan laddas datan in, och så skapar vi en variabel som är logaritmen av BNP per capita, `gle_rgdpc`.

In [1]:
use "https://www.qogdata.pol.gu.se/dataarchive/qog_bas_cs_jan18.dta", clear
gen ln_gdpc = ln(gle_rgdpc)


(Quality of Government Basic dataset 2018 - Cross-Section)

(2 missing values generated)


## Logaritmerad variabel som oberoende
Vi ska använda denna nya variabel som oberoende, och förväntad livslängd som beroende. Tanken är alltså att en högre BNP per capita hänger ihop med en längre förväntad livslängd - till exempel för att den högre nationalinkomsten kan användas till att förbättra grundläggande infrastruktur och sjukvård.


I koden nedan görs två analyser, med både faktisk BNP per capita och med den logaritmerade variabeln i varsin modell. Outputen undertrycks med hjälp av prefixet `quietly` framför regressionskommandot. Därefter sparas outputen med `estimates store` och presenteras slutligen tillsammans i en tabell med hjälp av kommandot `esttab` ([se separat guide](esttab_sv.html)), för att det ska bli lätt att jämföra analyserna.

In [6]:
quietly reg wdi_lifexp gle_rgdpc
estimates store m1

quietly reg wdi_lifexp ln_gdpc

estimates store m2
esttab m1 m2, r2







--------------------------------------------
                      (1)             (2)   
               wdi_lifexp      wdi_lifexp   
--------------------------------------------
gle_rgdpc        0.000346***                
                  (11.00)                   

ln_gdpc                             5.082***
                                  (18.37)   

_cons               67.17***        27.17***
                 (111.73)         (11.17)   
--------------------------------------------
N                     183             183   
R-sq                0.401           0.651   
--------------------------------------------
t statistics in parentheses
* p<0.05, ** p<0.01, *** p<0.001


Huvuddragen i de båda analyserna är givetvis desamma (det är trots allt samma variabel): det finns ett positivt och signifikant samband. Men vi ser att $R^2$-värdet är högre i den andra modellen, med den logaritmerade variabeln, vilket tyder på att den passar till datan bättre.

__Men hur ska man nu tolka koefficienterna?__ I modell 1, med den vanliga variabeln är det enkelt. Koefficienten visar vad som förväntas hända med den beroende variabeln om den oberoende variabeln ökar ett steg. Alltså: Om BNP per capita ökar med en dollar, förväntas livslängden öka med 0,000346 år. Inte så mycket, men så är en dollar inte heller så mycket.

I den andra modellen är tolkningen helt annorlunda. Rent tekniskt är det samma sak. Koefficienten visar att om vi ökar den logaritmerade variabeln med ett steg förväntas livslängden öka med 5,082 år. Men vad betyder det? För att omsätta det i mer konkreta termer kan vi dela koefficienten med 100, så att den blir 0,05082. __Det motsvarar den förväntade ökningen i livslängd, om vi ökar BNP per capita med en procent jämfört med vad den var tidigare.__

Varför är vi tvungna att dela koefficienten med hundra? Varför kan vi inte bara säga att livslängden förväntas öka med 5,082 år om vi ökar BNP per capita med 100 procent?

Svaret är att förändringstakten bara gäller vid en enskild punkt. Ju större steg vi tar, desto mindre träffsäkert blir det. För att öka den logaritmerade BNP per capita med ett helt steg måste det till mycket mer än en fördubbling.

## En liknelse
Anledningen är "ränta på ränta". Tänk dig att du har hundra kronor, som du ska sätta in på banken, och ha där i hundra dagar. Banken (det är en väldigt snäll bank) låter dig välja på följande ränteplaner:

Alternativ 1: 100 procents ränta var hundrade dag<br>
Alternativ 2: 10 procents ränta var tionde dag<br>
Alternativ 3: 1 procents ränta varje dag

Vilket är bäst? Det kan låta som att alternativen är likvärdiga. Men så är det inte. Med alternativ ett skulle du efter de hundra dagarna ha 200 kronor. Först händer inget i 99 dagar, och sedan får du en hundralapp till.

Med alternativ två skulle du efter tio dagar ha 110 kronor, och när du nästa gång får ränta beräknas räntan även på de tio extra kronorna. Efter hundra dagar skulle du då ha 259 kronor!

Och med alternativ tre blir det ännu fler tillfällen att få ränta på räntan, vilket gör att du efter de hundra dagarna skulle ha 270 kronor. Om du kunde få en ränta på en promille, utbetalad tio gånger om dagen, skulle du tjäna ytterligare en krona.

Att öka den förväntade livslängden med 5,082 år kräver alltså att vi ökar BNP per capita med 1 procent hundra gånger, eller om vi vill vara ännu mer noggranna, att vi ökar den med 1 promille 1000 gånger. I praktiken innebär det att vi behöver öka den så att den är 2,71828 gånger så stor som den var förut. Känns talet igen? Det är det naturliga talet $e$, som är basen till den naturliga logaritmen som vi använde för att konstruera variabeln.

## Logaritmerad variabel som beroende

Hur blir det då om man har en beroende variabel som är logaritmerad? Då får vi tänka lite annorlunda. Säg till exempel att vi vill undersöka hur logaritmerad BNP per capita hänger ihop med graden av korruption `ti_cpi`, där höga värden indikerar mindre korruption. Vi gör då följande regression (med undertryckt output, och istället visad med `esttab`).

In [26]:
quietly reg ln_gdpc ti_cpi
esttab




----------------------------
                      (1)   
                  ln_gdpc   
----------------------------
ti_cpi             0.0463***
                  (13.09)   

_cons               6.695***
                  (40.04)   
----------------------------
N                     179   
----------------------------
t statistics in parentheses
* p<0.05, ** p<0.01, *** p<0.001


Koefficienten är 0,0463. I det här fallet kan vi då multiplicera koefficienten med hundra för att få ut den förväntade procentuella förändringen i den beroende variabeln, om vi ökar den oberoende med ett steg.

Dvs, varje steg uppåt på `ti_cpi` hänger ihop med en ökning av BNP per capita med 4,63% jämfört med vad BNP per capita var tidigare.

## Logaritmerad variabel som både beroende och oberoende

Det allra enklaste fallet är när både beroende och oberoende variabel är logaritmerade. Då är det bara att tolka koefficienten som den förväntade procentuella förändringen i den beroende variabeln när den oberoende variabeln ökar med en procent. Till exempel om vi vill se sambandet mellan logaritmerad befolkning och logaritmerad BNP (alltså inte BNP per capita).

In [36]:
gen ln_gdp = ln(gle_gdp)
gen ln_pop = ln(gle_pop)

quietly reg ln_gdp ln_pop
esttab


(2 missing values generated)

(2 missing values generated)



----------------------------
                      (1)   
                   ln_gdp   
----------------------------
ln_pop              0.942***
                  (22.41)   

_cons               2.321***
                   (6.23)   
----------------------------
N                     192   
----------------------------
t statistics in parentheses
* p<0.05, ** p<0.01, *** p<0.001


En ökning av landets befolkning med en procent hänger ihop med en ökning av landets totala BNP med 0,942%. Busenkelt.

## Avslutning

Det är ofta befogat att använda logaritmerade variabler i regressionsanalyser, när datan är kontinuerlig men skev. Men det gäller att hålla tungan rätt i mun när man tolkar koefficienterna. Här är en tabell som visar tolkningen för fyra olika scenarion.


| Beroende | Oberoende | Tolkning av b-koefficienten |
|----------|-----------|----------|
| Vanlig   | Vanlig    | Hur många __skalsteg__ den beroende förändras när vi ökar den oberoende med __ett skalsteg__. |
| Vanlig   | Logaritmerad    | Dela koefficenten på hundra: Hur många __skalsteg__ den beroende förändras när vi ökar den oberoende med __en procent__. |
| Logaritmerad   | Vanlig    | Multiplicera koefficenten med hundra: Hur många __procent__ den beroende förändras när vi ökar den oberoende med __ett skalsteg__. |
| Logaritmerad   | Logaritmerad    | Hur många __procent__ den beroende förändras när vi ökar den oberoende med __en procent__. |